# Libraries

In [13]:
# General libraries for scientific computation
import numpy as np
import pandas as pd
import os
import warnings

# Graphic libraries
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import itertools
import plotly.plotly as py
import plotly.graph_objs as go

# Preprocessing
from sklearn.preprocessing import StandardScaler

# Model validation
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Feature extraction
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Other tools
from sklearn.pipeline import Pipeline
import time
#import string

# Visualize decision-tree
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
from IPython.display import display
import pydotplus


from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

from sklearn.model_selection import GridSearchCV

In [2]:
from IPython.display import Image
%matplotlib inline

# General funcions

- #### Plot confusion matrix

In [3]:
# This code has been take from the official documentation of scikit-learn and it was modified
# for the porpuses of the present work
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    plt.FormatStrFormatter('%.2f')
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:.2f}".format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

- #### Plot Learning Curve

In [4]:
def func_learning_curve(est, X_train, y_train, save_result=False):
    train_sizes, train_scores, test_scores = learning_curve(estimator=est, X=X_train, y=y_train,
                                               train_sizes=np.linspace(0.1, 1.0, 10), cv=10, n_jobs=-1)

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.plot(train_sizes, train_mean, color='blue', marker='o',
             markersize=5, label='training accuracy')

    plt.fill_between(train_sizes, train_mean + train_std,
                     train_mean - train_std, alpha=0.15, color='blue')

    plt.plot(train_sizes, test_mean, color='green', linestyle='--',
             marker='s', markersize=5, label='validation accuracy')

    plt.fill_between(train_sizes, test_mean + test_std,
                     test_mean - test_std, alpha=0.15, color='green')

    plt.grid()
    plt.xlabel('Number of training samples')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.ylim([0.0, 1.2])
    plt.tight_layout()

    if(save_result):
        plt.savefig('./../Results/learning_curve.png', dpi=300)

    plt.show()

- #### Plot Validation Curve

In [5]:
def func_validation_curve(est, X_train, y_train, param_name, param_range, scale=True, save_result=False):
    train_scores, test_scores = validation_curve(estimator=est, X=X_train, y=y_train, param_name=param_name, 
                                                     param_range=param_range, cv=10, n_jobs=-1)

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.plot(param_range, train_mean, 
             color='blue', marker='o', 
             markersize=5, label='training accuracy')

    plt.fill_between(param_range, train_mean + train_std,
                     train_mean - train_std, alpha=0.15,
                     color='blue')

    plt.plot(param_range, test_mean, 
             color='green', linestyle='--', 
             marker='s', markersize=5, 
             label='validation accuracy')

    plt.fill_between(param_range, 
                     test_mean + test_std,
                     test_mean - test_std, 
                     alpha=0.15, color='green')

    plt.grid()
    if(scale):
        plt.xscale('log')
    plt.legend(loc='lower right')
    plt.xlabel('Parameter ' + param_name)
    plt.ylabel('Accuracy')
    plt.ylim([0.0, 1.2])
    plt.tight_layout()
    
    if(save_result):
        plt.savefig('./../Results/validation_curve.png', dpi=300)
    
    plt.show()

# Dataset (Vehicle silhouettes)

link: http://archive.ics.uci.edu/ml/datasets/Statlog+%28Vehicle+Silhouettes%29

In [6]:
file = './../Data/metadata_vehicles.txt'
metadata = pd.read_table(file, sep=',', engine='python', header=0)

print('The model has', metadata.shape[0]-1, 'columns')
metadata

The model has 18 columns


,Original variable name,Formula,Column name in the model
0,COMPACTNESS,(average perim)**2/area,compactness
1,CIRCULARITY,(average radius)**2/area,circularity
2,DISTANCE CIRCULARITY,area/(av.distance from border)**2,distance_circularity
3,RADIUS RATIO,(max.rad-min.rad)/av.radius,radius_ratio
4,PR.AXIS ASPECT RATIO,(minor axis)/(major axis),pr_axis_aspect_ratio
5,MAX.LENGTH ASPECT RATIO,(length perp. max length)/(max length),max_length_aspect_ratio
6,SCATTER RATIO,(inertia about minor axis)/(inertia about majo...,scatter_ratio
7,ELONGATEDNESS,area/(shrink width)**2,elongatedness
8,PR.AXIS RECTANGULARITY,area/(pr.axis length*pr.axis width),pr_axis_rectangularity
9,MAX.LENGTH RECTANGULARITY,area/(max.length*length perp. to this),max_length_rectangularity


In [7]:
directory = './../Data/statlog-vehicle/'
vehicles = pd.DataFrame()

for f in os.listdir(directory):
    file = pd.read_table(directory+f, sep='\s', engine='python', header=None)
    vehicles = pd.concat([vehicles, file])

vehicles.columns = metadata['Column name in the model'].values
class_names=['bus', 'opel', 'saab', 'van']

In [12]:
data = []
for col in vehicles.columns:
    data.append(  go.Box( y=vehicles[col], name=col, showlegend=False ) )

data.append( go.Scatter( x = vehicles.columns, y = vehicles.mean(), mode='lines', name='mean' ) )

# IPython notebook
py.iplot(data, filename='pandas-box-plot')
#url = py.plot(data, filename='pandas-box-plot')


High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jaime9510/0 or inside your plot.ly account where it is named 'pandas-box-plot'


# Preprocessing

### Label decoding

In [ ]:
class_label = {label: idx for idx, label in enumerate(np.unique(vehicles['vehicle']))}
class_label

In [ ]:
vehicles.ix[:,18] = vehicles.ix[:,18].map(class_label)
print(vehicles.shape)
vehicles.head()

### Train - Test split

In [ ]:
X, y = vehicles.iloc[:, :-1].values, vehicles.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
print('X_train size', X_train.shape[0], '- X_test size', X_test.shape[0])

### Standardize

In [ ]:
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Feature Extraction

## Principal Component Analysis (PCA) 

In [ ]:
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
pca.explained_variance_ratio_

In [ ]:
plt.bar(range(1, 19), pca.explained_variance_ratio_, alpha=0.5, align='center')
plt.step(range(1, 19), np.cumsum(pca.explained_variance_ratio_), where='mid')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.savefig('./../Results/PCA.png', dpi=300)
plt.show()

In [ ]:
pca = PCA(n_components=9)
X_train_pca = pca.fit_transform(X_train_std)
print('%', pca.explained_variance_ratio_)
print('Varianza spiegata per 9 componenti', sum(pca.explained_variance_ratio_))

In [ ]:
X_test_pca = pca.transform(X_test_std)

## Linear Discriminant Analysis (LDA)

In [ ]:
lda = LDA(n_components=None)
X_train_lda = lda.fit_transform(X_train_std, y_train)

In [ ]:
print(lda.explained_variance_ratio_ )
print(sum(lda.explained_variance_ratio_ ))

In [ ]:
X_test_lda = lda.transform(X_test_std)

# Data for the model

In [ ]:
# (X_train_std, X_test_std) (X_train_pca, X_test_pca); (X_train_lda, X_test_lda)

X_train_model = X_train_std
X_test_model = X_test_std

# Logistic Regression

Only this model will be implement in two ways. The first one in which it is tested manually the cross-validation process and results; and the second one, that automates the process to find the best parameters for the model using also cross-validation

#### Cross-validation

In [ ]:
lr = LogisticRegression(C=100.0, random_state=0)

kfold = StratifiedKFold(n_splits=10,random_state=1).split(X_train_model, y_train)
#kfold = KFold(n_splits=10,random_state=1).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    lr.fit(X_train_model[train], y_train[train])
    score = lr.score(X_train_model[test], y_train[test])
    scores.append(score)
    print('Fold: %s, Accuracy: %.3f' % (k+1, score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
lr.fit(X_train_model, y_train)
print(X_train_model.shape)
print('Test accuracy: %.4f' % lr.score(X_test_model, y_test))

#### Pipelining

In [ ]:
pipe_lr = Pipeline([('lr', LogisticRegression(random_state=0))])

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'lr__C': param_range}]

gs = GridSearchCV(estimator=pipe_lr, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train_model, y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
pd.DataFrame(gs.cv_results_).to_csv(path_or_buf='./../Results/lr_results.csv', decimal=',')

In [ ]:
c_lr = gs.best_params_.get('lr__C', 1)
lr = LogisticRegression(C=c_lr, random_state=0)
lr.fit(X_train_model, y_train)

print('Train accuracy: %.4f' % lr.score(X_train_model, y_train))
print('Test accuracy: %.4f' % lr.score(X_test_model, y_test))

### Model analysis

- ##### Confusion matrix

In [ ]:
y_pred = lr.predict(X_test_model)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)

plt.figure()
plot_confusion_matrix(confmat, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.figure()
plot_confusion_matrix(confmat, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

- ##### Learning Curve

In [ ]:
func_learning_curve(lr, X_train_model, y_train, True)

- ##### Validation Curve

In [ ]:
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_name = 'C'
func_validation_curve(lr, X_train_model, y_train, param_name, param_range, True, True)

# Tree-Based Methods

## Decision Trees

In [ ]:
pipe_dt = Pipeline([('dt', DecisionTreeClassifier(random_state=0))])

param_range = [3, 5, 7, 10, 50, 100]

param_grid = [{'dt__max_depth': param_range}]

gs = GridSearchCV(estimator=pipe_dt, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train_model, y_train)
print('Best score: %.4f' % gs.best_score_)
print('Best params: %s' % gs.best_params_)

In [ ]:
pd.DataFrame(gs.cv_results_).to_csv(path_or_buf='./../Results/dt_results.csv', decimal=',')

In [ ]:
md = gs.best_params_.get('dt__max_depth', None)

tree = DecisionTreeClassifier(criterion='gini', max_depth=md, random_state=0)
tree.fit(X_train_model, y_train)

print('Train accuracy: %.4f' % tree.score(X_train_model, y_train))
print('Test accuracy: %.4f' % tree.score(X_test_model, y_test))

### Model analysis

- ##### Confusion matrix

In [ ]:
y_pred = tree.predict(X_test_model)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)

plt.figure()
plot_confusion_matrix(confmat, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.figure()
plot_confusion_matrix(confmat, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

- ##### Tree graph

In [ ]:
dot_data = export_graphviz(tree, out_file=None,
                feature_names = metadata['Column name in the model'].values,
                class_names=['bus', 'opel', 'saab', 'van'],
                filled=True, rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data) 
print(graph)
display(Image(graph.create_png()))


- ##### Learning Curve

In [ ]:
func_learning_curve(tree, X_train_model, y_train, True)

- ##### Validation Curve

In [ ]:
param_range = [3, 5, 7, 10, 30, 50, 75, 100]
param_name = 'max_depth'
func_validation_curve(tree, X_train_model, y_train, param_name, param_range, False, True)

## Random Forest

In [ ]:
pipe_rf = Pipeline([('rf', RandomForestClassifier(criterion='gini', random_state=1))])

param_range = [3, 5, 7, 10, 50, 100, 300, 500, 1000]

param_grid = [{'rf__n_estimators': param_range}]

gs = GridSearchCV(estimator=pipe_rf, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train_model, y_train)

print('Best score: %.4f' % gs.best_score_)
print('Best params: %s' % gs.best_params_)

In [ ]:
pd.DataFrame(gs.cv_results_).to_csv(path_or_buf='./../Results/rf_results.csv', decimal=',')

In [ ]:
ne = gs.best_params_.get('rf__n_estimators', None)
forest = RandomForestClassifier(criterion='gini',
                                n_estimators=ne, 
                                random_state=1,
                                n_jobs=-1)
forest.fit(X_train_model, y_train)

print('Train accuracy: %.4f' % forest.score(X_train_model, y_train))
print('Test accuracy: %.4f' % forest.score(X_test_model, y_test))

### Model analysis

- ##### Confusion matrix

In [ ]:
y_pred = forest.predict(X_test_model)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)

plt.figure()
plot_confusion_matrix(confmat, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.figure()
plot_confusion_matrix(confmat, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

- ##### Learning curve

In [ ]:
start_time = time.time()
func_learning_curve(forest, X_train_model, y_train, True)
print("--- %s seconds ---" % (time.time() - start_time))

- ##### Validation curve

In [ ]:
param_range = [3, 5, 7, 10, 50, 100, 300, 500, 1000]
param_name = 'n_estimators'
func_validation_curve(forest, X_train_model, y_train, param_name, param_range, False, True)

# Support Vector Machines (SVM)

### Linear kernel

In [ ]:
pipe_svm = Pipeline([('svm', SVC(random_state=1))])

param_range_C = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svm__C': param_range_C, 
               'svm__kernel': ['linear']}]

gs = GridSearchCV(estimator=pipe_svm, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train_model, y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
pd.DataFrame(gs.cv_results_).to_csv(path_or_buf='./../Results/svm_linear_results.csv', decimal=',')

In [ ]:
c = gs.best_params_.get('svc__C', 1)
svm = SVC(kernel='linear', C=c, random_state=1)
svm.fit(X_train_model, y_train)

print('Train accuracy: %.4f' % svm.score(X_train_model, y_train))
print('Test accuracy: %.4f' % svm.score(X_test_model, y_test))

### Model analysis

- ##### Confusion matrix

In [ ]:
y_pred = svm.predict(X_test_model)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)

plt.figure()
plot_confusion_matrix(confmat, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.figure()
plot_confusion_matrix(confmat, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

- ##### Learning curve

In [ ]:
start_time = time.time()
func_learning_curve(svm, X_train_model, y_train, True)
print("--- %s seconds ---" % (time.time() - start_time))

- ##### Validation curve

In [ ]:
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_name = 'C'
func_validation_curve(svm, X_train_model, y_train, param_name, param_range, True, True)

### Kernel rbf

In [ ]:
pipe_svm_rbf = Pipeline([('svm_rbf', SVC(random_state=1))])

param_range_gamma = [0.001, 0.01, 0.1, 1.0, 10.0]
param_range_c = [0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svm_rbf__C': param_range_c, 
                  'svm_rbf__gamma': param_range_gamma, 
                  'svm_rbf__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svm_rbf, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train_model, y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
c_rbf = gs.best_params_.get('svm_rbf__C', 1)
g = gs.best_params_.get('svm_rbf__gamma', 1)

svm_rbf = SVC(kernel='rbf', random_state=0, gamma=g, C=c_rbf)
svm_rbf.fit(X_train_model, y_train)

print('Train accuracy: %.4f' % svm_rbf.score(X_train_model, y_train))
print('Test accuracy: %.4f' % svm_rbf.score(X_test_model, y_test))

### Model analysis

- ##### Confusion matrix

In [ ]:
y_pred = svm_rbf.predict(X_test_model)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)

plt.figure()
plot_confusion_matrix(confmat, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.figure()
plot_confusion_matrix(confmat, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

- ##### Learning curve

In [ ]:
start_time = time.time()
func_learning_curve(svm_rbf, X_train_model, y_train, True)
print("--- %s seconds ---" % (time.time() - start_time))

- ##### Validation curve

In [ ]:
param_range = [0.1, 1.0, 10.0, 100.0, 1000.0]
param_name = 'C'
func_validation_curve(svm_rbf, X_train_model, y_train, param_name, param_range, True, True)

In [ ]:
param_range = [0.001, 0.01, 0.1, 1.0, 10.0]
param_name = 'gamma'
func_validation_curve(svm_rbf, X_train_model, y_train, param_name, param_range, True, True)

# Feature importance

- #### Using Random Forest

In [ ]:
rf_importance = pd.DataFrame(data=forest.feature_importances_, 
                             index=vehicles.ix[:,:-1].columns, columns=['Importance'])

rf_importance.sort_values(by=['Importance'], ascending=False)

- #### L1-Regularization

In [ ]:
lr = LogisticRegression(penalty='l1', C=0.1)
lr.fit(X_train_model, y_train)
print('Training accuracy:', lr.score(X_train_model, y_train))
print('Test accuracy:', lr.score(X_test_model, y_test))

In [ ]:
p = pd.DataFrame(lr.coef_)
vehicles.columns[p.columns[(p == 0).all()]]

In [ ]:
p